In [1]:
import pandas as pd

In [14]:
df = pd.read_csv('clean_tweets.csv')

/Users/laureleter/.pyenv/versions/lewagon/envs/datacraft/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
df.head()

,Unnamed: 0,Unnamed: 0.1,pseudo,tweet,tweet_date,tweet_url,tweet_hashtags,tweet_urls,tweet_mentions,status_id,...,name,location,description,followers_count,friends_count,listed_count,statuses_count,favourites_count,account_created_at,profile_expanded_url
0,0,0,(Mysteriously Unnamed),AiphanMarcel Quand imposera vaccin puce,2020-04-05 20:21:03.0,http://twitter.com//statuses/1246895646730260488,NaN,NaN,Cheewyy4| nonopp4,1.246896e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,_________adaa,Je explique soie disant vaccin intérieur nano ...,2020-11-12 15:03:13.0,http://twitter.com/_________adaa/statuses/1326...,NaN,NaN,umut_dkr,1.326903e+18,...,ADAA 💕🇸🇳,"Paris, France",De passage 🙃,729.0,431.0,0.0,3347.0,4165.0,2020-03-27 17:58:56,https://instagram.com/___.adaaa?igshid=1re6u03...
2,2,2,______Stan_____,RT laurentchemla a déjà réussi faire tourner D...,2021-01-13 14:37:39.0,http://twitter.com/______Stan_____/statuses/13...,NaN,NaN,laurentchemla,1.349365e+18,...,Stan,"Marseille, France",Dolce Vita à Marseille. \nDiffuseur de bon pla...,509.0,468.0,22.0,18153.0,11004.0,2014-10-26 13:18:03,NaN
3,3,3,_____Aurore,RT POITIERS nounou potit chat asso a débarqué ...,2020-10-13 05:07:59.0,http://twitter.com/_____Aurore/statuses/131588...,NaN,https://t.co/wNxLIcNzoh,Lola__bos,1.315882e+18,...,aurora 🇫🇷🇮🇹,Italy,twitter is dark and full of spoilers.,475.0,353.0,12.0,66194.0,2097.0,2012-01-07 17:21:43,https://unefrenchieenitalie.home.blog/
4,4,4,_____ilham,RT Moi quand gouvernement contrôlera esprit gr...,2020-11-13 00:09:38.0,http://twitter.com/_____ilham/statuses/1327040...,NaN,https://t.co/WGy1705Ie8,MikeStnm_,1.327041e+18,...,Chinup,NaN,Open up ur soul show me how its fake,37.0,94.0,0.0,1265.0,1443.0,2016-11-24 19:22:51,NaN


# Topic modeling test with gensim

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [102]:
pip install gensim

  Using cached gensim-4.0.1-cp38-cp38-macosx_10_9_x86_64.whl (23.9 MB)
     |████████████████████████████████| 57 kB 4.6 MB/s eta 0:00:011
You should consider upgrading via the '/Users/laureleter/.pyenv/versions/lewagon/envs/datacraft/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [147]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


/Users/laureleter/.pyenv/versions/lewagon/envs/datacraft/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [150]:
data = df.tweet.values.tolist()

In [152]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['aiphanmarcel', 'quand', 'imposera', 'vaccin', 'puce']]


In [153]:
# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]]


In [154]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aiphanmarcel', 1),
  ('imposera', 1),
  ('puce', 1),
  ('quand', 1),
  ('vaccin', 1)]]

In [156]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [158]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.209*"contre" + 0.064*"sou" + 0.060*"obligatoire" + 0.053*"afrique" + 0.043*"pfizer" + 0.036*"peau" + 0.035*"polio" + 0.035*"personnes" + 0.031*"comment" + 0.026*"injecte"'), (1, '0.402*"les" + 0.284*"covid" + 0.115*"faut" + 0.047*"maintenant" + 0.029*"nanoparticule" + 0.013*"gps" + 0.011*"ya" + 0.009*"envie" + 0.005*"injectez" + 0.005*"serieusement"'), (2, '0.243*"vaccin" + 0.108*"puce" + 0.055*"la" + 0.050*"plus" + 0.038*"monde" + 0.036*"va" + 0.035*"via" + 0.034*"fait" + 0.028*"non" + 0.023*"tous"'), (3, '0.184*"bien" + 0.144*"vaccine" + 0.119*"tres" + 0.075*"an" + 0.070*"bon" + 0.067*"grand" + 0.058*"ou" + 0.051*"pas" + 0.037*"porte" + 0.035*"entier"'), (4, '0.418*"nanoparticules" + 0.120*"mais" + 0.084*"controle" + 0.063*"complotiste" + 0.052*"celui" + 0.037*"realite" + 0.034*"debut" + 0.032*"compris" + 0.015*"electrique" + 0.015*"schema"'), (5, '0.219*"si" + 0.103*"encore" + 0.094*"des" + 0.078*"nous" + 0.040*"milliardaire" + 0.035*"donne" + 0.031*"adn" + 0.029*"nom" + 0.0

In [160]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -16.8402245918738

Coherence Score:  0.28115348065629286


In [168]:
num_topics = 10
#lda = lda_model(corpus=corpus, id2word=id2word, num_topics=num_topics, passes=20, random_state=0)
for i in range(num_topics):
    print(f"********** Topic {i+1} **********")
    print(lda_model.print_topic(i), '\n')


********** Topic 1 **********
0.209*"contre" + 0.064*"sou" + 0.060*"obligatoire" + 0.053*"afrique" + 0.043*"pfizer" + 0.036*"peau" + 0.035*"polio" + 0.035*"personnes" + 0.031*"comment" + 0.026*"injecte" 

********** Topic 2 **********
0.402*"les" + 0.284*"covid" + 0.115*"faut" + 0.047*"maintenant" + 0.029*"nanoparticule" + 0.013*"gps" + 0.011*"ya" + 0.009*"envie" + 0.005*"injectez" + 0.005*"serieusement" 

********** Topic 3 **********
0.243*"vaccin" + 0.108*"puce" + 0.055*"la" + 0.050*"plus" + 0.038*"monde" + 0.036*"va" + 0.035*"via" + 0.034*"fait" + 0.028*"non" + 0.023*"tous" 

********** Topic 4 **********
0.184*"bien" + 0.144*"vaccine" + 0.119*"tres" + 0.075*"an" + 0.070*"bon" + 0.067*"grand" + 0.058*"ou" + 0.051*"pas" + 0.037*"porte" + 0.035*"entier" 

********** Topic 5 **********
0.418*"nanoparticules" + 0.120*"mais" + 0.084*"controle" + 0.063*"complotiste" + 0.052*"celui" + 0.037*"realite" + 0.034*"debut" + 0.032*"compris" + 0.015*"electrique" + 0.015*"schema" 

********** Topi

# Test assign KapCode topics on each tweet with classification model

In [206]:

topics = pd.read_excel("topics_per_tweet.7topics.xlsx", engine="openpyxl")

In [207]:
topics.head()

,status_id,topic_id,Label,tweet
0,1235030805128155138,0.0,NaN,Outrageous.
1,1262796554605297665,0.0,NaN,@BFMTV Vous nous suggérez alors le vaccin de B...
2,1358050076925583360,0.0,NaN,"@slessard2000 Se moquer, c'est pas bien. https..."
3,1351200956625670150,0.0,NaN,@SFR Vous avez détourné les vaccins de Bill Gâ...
4,1334807406568747008,0.0,NaN,ET VOIL0 LE VACCIN DE BILL GATES


In [208]:
topics.Label = topics.Label.astype(str)

In [209]:
topics.tweet = topics.tweet.astype(str)

In [210]:
topics.Label.value_counts()

Incrédules                                  25762
Ordre mondial, contrôle de la population    19637
Action de Bill Gates en Afrique             10848
Financement du vaccin                        7968
Animaux et crédules                          1729
Production du vaccin                         1335
Qanon                                        1281
nan                                           332
Name: Label, dtype: int64

In [211]:
topics.topic_id.value_counts()

1.0    25762
7.0    19637
4.0    10848
3.0     7968
5.0     1729
2.0     1335
6.0     1281
0.0       86
Name: topic_id, dtype: int64

In [214]:
topics.topic_id = topics.topic_id.fillna(0.0).astype(int)

In [215]:
topics.topic_id.value_counts()

1    25762
7    19637
4    10848
3     7968
5     1729
2     1335
6     1281
0      332
Name: topic_id, dtype: int64

In [236]:
topics['target_name'] = topics.topic_id.map({0:'No topic', 1: 'Incredules', 2:"Production du vaccin", 3:"Financement du vaccin", 4:"Action de Bill Gates en Afrique", 5:'Animaux et credules', 6:"Qanon", 7:"Ordre mondial, Contrôle de la population"})

In [202]:
topics.topic_id.value_counts()

1    25762
7    19637
4    10848
3     7968
5     1729
2     1335
6     1281
0      332
Name: topic_id, dtype: int64

In [ ]:
import gensim

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus,
                           num_topics=20,
                           id2word=train_id2word,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train.save('lda_train.model')

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components=1)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)


In [185]:
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords 
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from joblib import dump, load
import pickle

In [216]:
# Holdout
X = topics[['tweet']]

y = topics['topic_id']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 1)

In [237]:
topics


,status_id,topic_id,Label,tweet,taget_name,target_name
0,1235030805128155138,0,nan,Outrageous.,No topic,No topic
1,1262796554605297665,0,nan,@BFMTV Vous nous suggérez alors le vaccin de B...,No topic,No topic
2,1358050076925583360,0,nan,"@slessard2000 Se moquer, c'est pas bien. https...",No topic,No topic
3,1351200956625670150,0,nan,@SFR Vous avez détourné les vaccins de Bill Gâ...,No topic,No topic
4,1334807406568747008,0,nan,ET VOIL0 LE VACCIN DE BILL GATES,No topic,No topic
...,...,...,...,...,...,...
68887,1249236275519016964,0,nan,@lemondefr Des vaccins 💉 et des puces 👾 pour t...,No topic,No topic
68888,1255284258366685184,0,nan,"@marienoellem Alors ""Gates"" toi avec le vaccin...",No topic,No topic
68889,1346143958880169985,0,nan,@DocPepperFR La puce 5 G du vaccin ?,No topic,No topic
68890,1332794028677607429,0,nan,On Gates vaccines 👇👇,No topic,No topic


In [229]:
target_names={0:'No topic', 1: 'Incredules', 2:"Production du vaccin", 3:"Financement du vaccin", 4:"Action de Bill Gates en Afrique", 5:'Animaux et credules', 6:"Qanon", 7:"Ordre mondial, Contrôle de la population"}

In [238]:
target_names = {n: i for i, n in enumerate(sorted(topics['target_name'].unique()))}

In [239]:
# Pipeline with french stop_words nltk in tfidf
nltk.download('stopwords')
stop_words = set(stopwords.words('french'))
ridge_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 2))),
                ('Ridge', RidgeClassifier()),
            ])
# train the model using X_dtm & y
ridge_pipeline.fit(X_train.squeeze(), y_train)
# Squeeze is to get the correct shape for X (if none I have a 1 column after my rows)
# compute the testing accuracy
prediction = ridge_pipeline.predict(X_test.squeeze())
print('Test accuracy is {}'.format(accuracy_score(y_test, prediction)))
print(classification_report(y_test, prediction, target_names=target_names))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/laureleter/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Test accuracy is 0.8042868202051481
                                          precision    recall  f1-score   support

         Action de Bill Gates en Afrique       0.73      0.08      0.15        96
                     Animaux et credules       0.79      0.89      0.84      7876
                   Financement du vaccin       0.92      0.87      0.90       394
                              Incredules       0.85      0.69      0.76      2343
                                No topic       0.82      0.75      0.79      3190
Ordre mondial, Contrôle de la population       0.96      0.53      0.68       509
                    Production du vaccin       0.97      0.83      0.90       362
                                   Qanon       0.77      0.79      0.78      5898

                                accuracy                           0.80     20668
                               macro avg       0.85      0.68      0.72     20668
                            weighted avg       0.81      0.8

In [240]:
# cross_validation
from sklearn.model_selection import cross_validate

# 5-Fold Cross validate model
cv_results = cross_validate(ridge_pipeline, X_train.squeeze(), y_train, cv=5)

# Scores
cv_results['test_score']

# Mean of scores
cv_score = cv_results['test_score'].mean()

cv_score

0.794749333396978

In [241]:
model = ridge_pipeline.fit(X_train.squeeze(), y_train)

In [243]:
import joblib

In [244]:
# Export fited model joblib format
joblib.dump(model, 'topics_model.joblib') 

['topics_model.joblib']

In [246]:
#to load jobil model
loaded_model = joblib.load('topics_model.joblib')
#result = loaded_model.score(X_test.squeeze(), y_test)
result = loaded_model.predict([topics['tweet'][0]])

In [251]:
df['tweet'] = df['tweet'].astype(str)

In [252]:
df['topics_prediction'] = loaded_model.predict(df['tweet'])

In [254]:
df

,Unnamed: 0,Unnamed: 0.1,pseudo,tweet,tweet_date,tweet_url,tweet_hashtags,tweet_urls,tweet_mentions,status_id,...,description,followers_count,friends_count,listed_count,statuses_count,favourites_count,account_created_at,profile_expanded_url,position_gps,topics_prediction
0,0,0,(Mysteriously Unnamed),AiphanMarcel Quand imposera vaccin puce,2020-04-05 20:21:03.0,http://twitter.com//statuses/1246895646730260488,NaN,NaN,Cheewyy4| nonopp4,1.246896e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,7
1,1,1,_________adaa,Je explique soie disant vaccin intérieur nano ...,2020-11-12 15:03:13.0,http://twitter.com/_________adaa/statuses/1326...,NaN,NaN,umut_dkr,1.326903e+18,...,De passage 🙃,729.0,431.0,0.0,3347.0,4165.0,2020-03-27 17:58:56,https://instagram.com/___.adaaa?igshid=1re6u03...,None,1
2,2,2,______Stan_____,RT laurentchemla a déjà réussi faire tourner D...,2021-01-13 14:37:39.0,http://twitter.com/______Stan_____/statuses/13...,NaN,NaN,laurentchemla,1.349365e+18,...,Dolce Vita à Marseille. \nDiffuseur de bon pla...,509.0,468.0,22.0,18153.0,11004.0,2014-10-26 13:18:03,NaN,None,1
3,3,3,_____Aurore,RT POITIERS nounou potit chat asso a débarqué ...,2020-10-13 05:07:59.0,http://twitter.com/_____Aurore/statuses/131588...,NaN,https://t.co/wNxLIcNzoh,Lola__bos,1.315882e+18,...,twitter is dark and full of spoilers.,475.0,353.0,12.0,66194.0,2097.0,2012-01-07 17:21:43,https://unefrenchieenitalie.home.blog/,None,4
4,4,4,_____ilham,RT Moi quand gouvernement contrôlera esprit gr...,2020-11-13 00:09:38.0,http://twitter.com/_____ilham/statuses/1327040...,NaN,https://t.co/WGy1705Ie8,MikeStnm_,1.327041e+18,...,Open up ur soul show me how its fake,37.0,94.0,0.0,1265.0,1443.0,2016-11-24 19:22:51,NaN,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193213,193213,193204,ZzNpl,RT Louis Fouché Les nanoparticules Vaccins plu...,2020-12-10 11:14:34.0,http://twitter.com/ZzNpl/statuses/133699270150...,Libertedexpression|LoiSecuriteGlobale|5G|COVID...,https://t.co/khSDyzp8lG,Mensongecovid19,1.336993e+18,...,Entre la NAÏVETÉ totale et le COMPLOTISME il y...,7.0,165.0,0.0,150.0,293.0,2020-11-11 18:40:49,NaN,None,3
193214,193214,193205,ZzNpl,RT afpfr Cette histoire vaccination obligatoir...,2020-12-01 20:06:34.0,http://twitter.com/ZzNpl/statuses/133386509120...,NaN,NaN,Sibir05132651,1.333865e+18,...,Entre la NAÏVETÉ totale et le COMPLOTISME il y...,7.0,165.0,0.0,150.0,293.0,2020-11-11 18:40:49,NaN,None,7
193215,193215,193206,ZzNpl,RT Le président Donald Trump a annulé projet B...,2020-11-20 07:06:39.0,http://twitter.com/ZzNpl/statuses/132968255247...,NaN,https://t.co/nQLTvkzTub,brunolp30| Carpesos,1.329683e+18,...,Entre la NAÏVETÉ totale et le COMPLOTISME il y...,7.0,165.0,0.0,150.0,293.0,2020-11-11 18:40:49,NaN,None,7
193216,193216,193207,ZzNpl,RT Alvisyma Ils peuvent introduire nano partic...,2020-12-02 11:52:40.0,http://twitter.com/ZzNpl/statuses/133410318614...,NaN,https://t.co/mnh486qvNE,Alvisyma,1.334103e+18,...,Entre la NAÏVETÉ totale et le COMPLOTISME il y...,7.0,165.0,0.0,150.0,293.0,2020-11-11 18:40:49,NaN,None,3


In [259]:
df['label'] = df.topics_prediction.map({0:'No topic', 1: 'Incredules', 2:"Production du vaccin", 3:"Financement du vaccin", 4:"Action de Bill Gates en Afrique", 5:'Animaux et credules', 6:"Qanon", 7:"Ordre mondial, Contrôle"})

In [260]:
df.head()

,Unnamed: 0,Unnamed: 0.1,pseudo,tweet,tweet_date,tweet_url,tweet_hashtags,tweet_urls,tweet_mentions,status_id,...,followers_count,friends_count,listed_count,statuses_count,favourites_count,account_created_at,profile_expanded_url,position_gps,topics_prediction,label
0,0,0,(Mysteriously Unnamed),AiphanMarcel Quand imposera vaccin puce,2020-04-05 20:21:03.0,http://twitter.com//statuses/1246895646730260488,NaN,NaN,Cheewyy4| nonopp4,1.246896e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,7,"Ordre mondial, Contrôle"
1,1,1,_________adaa,Je explique soie disant vaccin intérieur nano ...,2020-11-12 15:03:13.0,http://twitter.com/_________adaa/statuses/1326...,NaN,NaN,umut_dkr,1.326903e+18,...,729.0,431.0,0.0,3347.0,4165.0,2020-03-27 17:58:56,https://instagram.com/___.adaaa?igshid=1re6u03...,None,1,Incredules
2,2,2,______Stan_____,RT laurentchemla a déjà réussi faire tourner D...,2021-01-13 14:37:39.0,http://twitter.com/______Stan_____/statuses/13...,NaN,NaN,laurentchemla,1.349365e+18,...,509.0,468.0,22.0,18153.0,11004.0,2014-10-26 13:18:03,NaN,None,1,Incredules
3,3,3,_____Aurore,RT POITIERS nounou potit chat asso a débarqué ...,2020-10-13 05:07:59.0,http://twitter.com/_____Aurore/statuses/131588...,NaN,https://t.co/wNxLIcNzoh,Lola__bos,1.315882e+18,...,475.0,353.0,12.0,66194.0,2097.0,2012-01-07 17:21:43,https://unefrenchieenitalie.home.blog/,None,4,Action de Bill Gates en Afrique
4,4,4,_____ilham,RT Moi quand gouvernement contrôlera esprit gr...,2020-11-13 00:09:38.0,http://twitter.com/_____ilham/statuses/1327040...,NaN,https://t.co/WGy1705Ie8,MikeStnm_,1.327041e+18,...,37.0,94.0,0.0,1265.0,1443.0,2016-11-24 19:22:51,NaN,None,1,Incredules


In [261]:
df['topics_words'] = df.topics_prediction.map({0:'No topic', 1: '5g, nanoparticules, comme, gens, injecter, controler, monde, virus, complot, mettre', 2:"ls, pharma, big, loi, virus, memes, blackrock, avia, kennedy, jaunes", 3:"nanoparticules, microsoft, 5g, adn, arn, passeport, numerique, comme, peau, technologie", 4:"fondation, enfants, oms, afrique, monde, finance, millions, polio, inde, sante", 5:'comme, vendre, pourront, peau, acheter, faudra, ecrit, être, attention, electroniques', 6:"pucerfid, youtube, control, mind, oligarchie, 5g, pedophilie, ue, pedocriminalite, op5g", 7:"oms, comme, monde, mondial, population, macron, virus, 5g, mondiale, obligatoire"})

In [262]:
df.head()

,Unnamed: 0,Unnamed: 0.1,pseudo,tweet,tweet_date,tweet_url,tweet_hashtags,tweet_urls,tweet_mentions,status_id,...,friends_count,listed_count,statuses_count,favourites_count,account_created_at,profile_expanded_url,position_gps,topics_prediction,label,topics_words
0,0,0,(Mysteriously Unnamed),AiphanMarcel Quand imposera vaccin puce,2020-04-05 20:21:03.0,http://twitter.com//statuses/1246895646730260488,NaN,NaN,Cheewyy4| nonopp4,1.246896e+18,...,NaN,NaN,NaN,NaN,NaN,NaN,None,7,"Ordre mondial, Contrôle","oms, comme, monde, mondial, population, macron..."
1,1,1,_________adaa,Je explique soie disant vaccin intérieur nano ...,2020-11-12 15:03:13.0,http://twitter.com/_________adaa/statuses/1326...,NaN,NaN,umut_dkr,1.326903e+18,...,431.0,0.0,3347.0,4165.0,2020-03-27 17:58:56,https://instagram.com/___.adaaa?igshid=1re6u03...,None,1,Incredules,"5g, nanoparticules, comme, gens, injecter, con..."
2,2,2,______Stan_____,RT laurentchemla a déjà réussi faire tourner D...,2021-01-13 14:37:39.0,http://twitter.com/______Stan_____/statuses/13...,NaN,NaN,laurentchemla,1.349365e+18,...,468.0,22.0,18153.0,11004.0,2014-10-26 13:18:03,NaN,None,1,Incredules,"5g, nanoparticules, comme, gens, injecter, con..."
3,3,3,_____Aurore,RT POITIERS nounou potit chat asso a débarqué ...,2020-10-13 05:07:59.0,http://twitter.com/_____Aurore/statuses/131588...,NaN,https://t.co/wNxLIcNzoh,Lola__bos,1.315882e+18,...,353.0,12.0,66194.0,2097.0,2012-01-07 17:21:43,https://unefrenchieenitalie.home.blog/,None,4,Action de Bill Gates en Afrique,"fondation, enfants, oms, afrique, monde, finan..."
4,4,4,_____ilham,RT Moi quand gouvernement contrôlera esprit gr...,2020-11-13 00:09:38.0,http://twitter.com/_____ilham/statuses/1327040...,NaN,https://t.co/WGy1705Ie8,MikeStnm_,1.327041e+18,...,94.0,0.0,1265.0,1443.0,2016-11-24 19:22:51,NaN,None,1,Incredules,"5g, nanoparticules, comme, gens, injecter, con..."


In [263]:
df.to_csv('vacin_tweets_with_topics')